In [1]:
// To see less warnings
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

Intitializing Scala interpreter ...

Spark Web UI available at http://74e4be6e0134:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1651715065741)
SparkSession available as 'spark'


import org.apache.log4j._


In [2]:
// Logistic Regression
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.sql.SparkSession


In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("LinearRegExample")
                        .config("spark.master", "local")
                        .getOrCreate()

import spark.implicits._

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7aad501e
import spark.implicits._


In [6]:
// Use Spark to read in the Titanic csv file.
val data = spark.read.option("header","true").option("inferSchema","true").format("csv").load("titanic.csv")

data: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [8]:
// Setting Up DataFrame for Machine Learning

// Grab only the columns we want
val logregdataall = data.select(data("Survived").as("label"), $"Pclass", $"Sex", $"Age", $"SibSp", $"Parch", $"Fare", $"Embarked")
val logregdata = logregdataall.na.drop()


// Deal with the Categorical columns

// Import VectorAssembler and Vectors
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors

// Deal with Categorical Columns

val genderIndexer = new StringIndexer().setInputCol("Sex").setOutputCol("SexIndex")
val embarkIndexer = new StringIndexer().setInputCol("Embarked").setOutputCol("EmbarkIndex")

val genderEncoder = new OneHotEncoder().setInputCol("SexIndex").setOutputCol("SexVec")
val embarkEncoder = new OneHotEncoder().setInputCol("EmbarkIndex").setOutputCol("EmbarkVec")

// Assemble everything together to be ("label","features") format
val assembler = (new VectorAssembler()
                  .setInputCols(Array("Pclass", "SexVec", "Age","SibSp","Parch","Fare","EmbarkVec"))
                  .setOutputCol("features") )



// Split the Data
val Array(training, test) = logregdata.randomSplit(Array(0.7, 0.3), seed = 12345)


// Set Up the Pipeline
import org.apache.spark.ml.Pipeline

val lr = new LogisticRegression()

val pipeline = new Pipeline().setStages(Array(genderIndexer,embarkIndexer,genderEncoder,embarkEncoder,assembler, lr))

// Fit the pipeline to training documents.
val model = pipeline.fit(training)

// Get Results on Test Set
val results = model.transform(test)


// MODEL EVALUATION

// For Metrics and Evaluation
import org.apache.spark.mllib.evaluation.MulticlassMetrics

// Need to convert to RDD to use this
val predictionAndLabels = results.select($"prediction",$"label").as[(Double, Double)].rdd

// Instantiate metrics object
val metrics = new MulticlassMetrics(predictionAndLabels)

// Confusion matrix
println("Confusion matrix:")
println(metrics.confusionMatrix)


Confusion matrix:
107.0  13.0  
18.0   62.0  


logregdataall: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]
logregdata: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors
genderIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_dba920215e26
embarkIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_023965352fe6
genderEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_2e637c7bb164
embarkEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_6d21831d65e4
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_49d6f2f78a23, handleInvalid=error, numInputCols=7
train...
